In [5]:
###############################################################################################################################################################

#Authors: Mbani Benson, Mercy Mwaniki, Felix Mutua
 
#Organization: Jomo Kenyatta University of Agriculture and Technology

#Dste: February 8th, 2019

#Description:

#This piece of code implements the Bruun Rule from within ArcMap. The Bruun Rule is a formula for estimating the magnitude of the retreat of the shoreline of a sandy shore in response to changes in sea level. The Bruun Rule gives a linear relationship between sea level rise and shoreline recession based on equilibrium profile theory, which asserts that shore face profile maintains an equilibrium shape, and as sea level rises the increasing accommodation space forces this equilibrium profile landward and upward to preserve its shape relative to the new sea level. As such, the Bruun Rule analysis assumes that the upper beach is eroded as the shore profile moves landward, and that the volume of eroded material is deposited offshore, resulting in a rise of the nearshore bottom which maintains a constant water depth

################################################################################################################################################################

import arcpy

import math

arcpy.env.workspace = "D:/hycristal"

arcpy.env.overwriteOutput = True

path = "D:/hycristal/"

class shorelineRetreat ():
    '''This class contains procedures and routines that together implement the Bruun Rule from an ArcGIS environment'''
    
    def __init__(self, H, T, shoreProfiles, DEM, seaLevel, referenceShoreline):
        '''H - non breaking significant wave height; T - Associated wave period; g - acceleration due to gravity'''
        self.H = float(H)
        self.T = float(T)
        self.shoreProfiles = shoreProfiles
        self.DEM = DEM
        self.seaLevel = float(seaLevel)
        self.referenceShoreline = referenceShoreline
        
       

    def closureDepth (self):
        '''Compute the closure depth and add it to the attribute table; d_min - minimum depth of closure'''
        #self.d_min = 2.28*self.H - 68.5*( math.pow(self.H , 2) / 9.80665 * math.pow(self.T , 2) )
        self.d_min = 1118
        arcpy.AddField_management(self.shoreProfiles, "DoC", "DOUBLE")
        self.cursor = arcpy.da.UpdateCursor(self.shoreProfiles, ["DoC"])
        for row in self.cursor:
            row[0] = self.d_min
            #row[0] = 1118
            self.cursor.updateRow(row)

        arcpy.ContourList_3d(self.DEM, path+"DoC_Contour_raw.shp", '"' + str(self.d_min) + '"')

        arcpy.Dissolve_management(path+"DoC_Contour_raw.shp", path+"DoC_Contour.shp","", "", "SINGLE_PART", "DISSOLVE_LINES")

        
    def activeProfileLength (self):
        '''Determine length of active profile and write to attribute table'''
        #arcpy.ExtendLine_edit(self.shoreProfiles, "", "FEATURE")
        #arcpy.TrimLine_edit(self.shoreProfiles, "", "DELETE_SHORT")
        arcpy.Intersect_analysis ([self.shoreProfiles, path+"DoC_Contour.shp"], path+"DoCProfilePts.shp", "ALL", "", "POINT")
        arcpy.SplitLineAtPoint_management(self.shoreProfiles, path+"DoCProfilePts.shp", path+"activeProfile.shp","")
        arcpy.AddField_management(path+"activeProfile.shp", "ActiveLen", "DOUBLE")
        self.cursor = arcpy.da.UpdateCursor(path+"activeProfile.shp", ["SHAPE@LENGTH","ActiveLen"])
        for row in self.cursor:
            row[1] = row[0]
            self.cursor.updateRow(row)


    def stackProfile(self):
        '''Generate 3D profiles'''
        arcpy.StackProfile_3d(path+"activeProfile.shp", self.DEM, path+"profile_values.dbf", path+"Surface_Profile")


    def bermHgt(self):
        '''Determine the height of the berm'''
        arcpy.AddField_management(path+"activeProfile.shp", "bermHgt", "DOUBLE")
        self.cursor = arcpy.da.UpdateCursor(path+"activeProfile.shp", ["bermHgt"])
        for row in self.cursor:
            row[0] = 16.4
            self.cursor.updateRow(row)

    def shorelineRecession(self):
        '''Finally determine the shoreline recession for each profile and plot these'''
        arcpy.AddField_management(path+"activeProfile.shp", "recession", "DOUBLE")
        self.cursor = arcpy.da.UpdateCursor(path+"activeProfile.shp", ["recession","bermHgt","ActiveLen","DoC"])
        for row in self.cursor:
            row[0] = ( row[2]/(row[1]+row[3]) ) * self.seaLevel
            self.cursor.updateRow(row)


    def newShoreline(self):
        '''Generate new shorelines. intersectionPoints is the point of the reference shoreline and the shore profiles'''
        arcpy.Intersect_analysis ([path+"activeProfile.shp", self.referenceShoreline], path+"intersectionPts.shp", "ALL", "", "POINT")
        arcpy.CopyFeatures_management(path+"intersectionPts.shp", path+"intersetionPoints.shp")
        self.cursor = arcpy.da.UpdateCursor(path+"intersetionPoints.shp",["SHAPE@XY","Azimuth", "recession"])
        for row in self.cursor:
            x,y = row[0]
            azimuthRadians = math.radians(row[1])
            newX = row[2]*math.sin(azimuthRadians)
            newY = row[2]*math.cos(azimuthRadians)
            row[0] = newX,newY
            self.cursor.updateRow(row)
            
        #arcpy.MakeFeatureLayer_management("intersetionPoints.shp", "intersetionPoints_lyr.lyr")
            
        #arcpy.PointsToLine_management("intersetionPoints_lyr.lyr", "newShore.shp")
        

waveHeight = arcpy.GetParameterAsText(0)
wavePeriod = arcpy.GetParameterAsText(1)
profile = arcpy.GetParameterAsText(2)
elevation = arcpy.GetParameterAsText(3)
lakeLevel = arcpy.GetParameterAsText(4)
shoreline2018 = arcpy.GetParameterAsText(5)

hycristal = shorelineRetreat(waveHeight,wavePeriod,profile,elevation,lakeLevel,shoreline2018)

hycristal.closureDepth()
hycristal.activeProfileLength()
hycristal.stackProfile()
hycristal.bermHgt()
hycristal.shorelineRecession()
hycristal.newShoreline()





        
    
        
        
        

        
        

                
    

    

    


2.82842712475
4
6
5
7
